# Long candle strategy  EDA

Long candle strategy is classification model.
Features: prices
Targets: -1,0,1 signal

To install conda and binance packages to this notebook uncomment the code below

In [10]:
%%capture
#%conda install -c plotly plotly=5.9.0
#%conda install pip
#%conda install twisted
%pip install plotly==5.9.0
%pip install twisted
%pip install pandas
#%pip install huobi-sdk==2.3.3

import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', None)

In [6]:
#mode="dev" # dev - work with local dev data
mode="yc" # yc - work with data from yandex cloud bots

symbol="BTC-USDT"
strategy="LongCandleDenseStrategy"

In [9]:

import os

# Downloading latest pytrade2 data
if mode == "yc":
    os.system("cd ./../deploy/yandex_cloud; ./download_data.sh")
    data_dir=f"../data/yandex-cloud"
else:
    data_dir=f"../data/dev"
    
print(f"Download completed. Local data dir: {data_dir}")

    

deploy_lib.sh: line 6: yc: command not found
INFO: Retrieving list of remote files for s3://pytrade2/data/ ...


Sync pytrade2 data from s3://pytrade2/data to /home/dima/projects/pytrade2/deploy/yandex_cloud/../../data/yandex-cloud
LongCandleDenseStrategy last opened: , closed: 
LSTMStrategy last opened: 2023-08-01 15:34:15.711000, closed: 2023-08-01 15:43:49.187000
LSTMStrategy2 last opened: , closed: 
SimpleKerasStrategy last opened: , closed: 
Current time Сб 25 ноя 2023 22:43:18 UTC
Download completed. Local data dir: ../data/yandex-cloud



!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
    An unexpected error has occurred.
  Please try reproducing the error using
  the latest s3cmd code from the git master
  branch found at:
    https://github.com/s3tools/s3cmd
  and have a look at the known issues list:
    https://github.com/s3tools/s3cmd/wiki/Common-known-issues-and-their-solutions-(FAQ)
  If the error persists, please report the
  following lines (removing any private
  info as necessary) to:
   s3tools-bugs@lists.sourceforge.net


!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

Invoked as: /home/dima/.local/bin/s3cmd sync -v s3://pytrade2/data/ /home/dima/projects/pytrade2/deploy/yandex_cloud/../../data/yandex-cloud/
Problem: <class 'TypeError: sequence item 1: expected str instance, bytes found
S3cmd:   2.3.0
python:   3.12.0 (main, Oct 21 2023, 17:44:38) [GCC 9.4.0]
environment LANG=en_US.UTF-8

Traceback (most recent call last):
  File "/home/dima/.local/bin/s3cmd", line 3286, in <module>
    rc = main()
         ^^^^^^
 

## Account

In [4]:
import plotly.express as px
import seaborn as sns
from matplotlib import pyplot as plt
import seaborn as sns

plot_figsize=(10,5)
sns.set_style("darkgrid")



def get_balance(strategy: str):
    """ Read balance history from data"""
    
    data_dir=f"../data"
    env_dir="yandex-cloud"
    account_dir=f"{data_dir}/{env_dir}/{strategy}/account"
    print(f"Reading balance files from {account_dir}")
    #file_path = sorted([f for f in os.listdir(account_dir) if f.endswith(f"{symbol}_{name}.csv")])[-1]
    files = sorted([f for f in os.listdir(account_dir) if f.endswith(f".csv")])
    df = pd.concat([pd.read_csv(f"{account_dir}/{f}", parse_dates=True, index_col="time") for f in files]).sort_index()
    df["datetime"] = df.index
    return df

balance = get_balance(strategy)
balance["balance"].plot()
plt.show()
px.line(balance, "datetime", "balance", title=f"{strategy} balance USDT").update_traces(mode='lines+markers').show()


ModuleNotFoundError: No module named 'seaborn'

## Predicted and actual low/high

In [ ]:
import os
import glob
import pandas as pd
from datetime import timedelta,date
import matplotlib.pyplot as plt
import plotly.express as px

def read_last(symbol: str, name: str, n=1000):
    xy_dir=f"{data_dir}/{strategy}/Xy"

    file_path = sorted([f for f in os.listdir(xy_dir) if f.endswith(f"{symbol}_{name}.csv")])[-1]
    file_path = f"{xy_dir}/{file_path}"
    print(f"Read from {file_path}")
    return pd.read_csv(file_path, parse_dates=True).tail(n)


def read_last_xy(data_dir):
    """ Read days from start to end from local predictions history"""
    
    #data=read_last(symbol, "data", n)
    X=read_last(symbol, "x")
    y=read_last(symbol, "y").set_index('close_time')
    data=pd.merge(X,y, left_index=True, right_index=True) 
    #y=pd.concat([data,y])
    return data,X,y


# Read candles for last 3 days
print(symbol)
data,x,y = read_last_xy(data_dir)
y.tail()


In [ ]:

#actual_signal_counts = y['signal_actual'].diff().to_numpy().nonzero().value_counts()
actual_signal_counts = y['signal_actual'].diff().value_counts()


#actual_signal_counts = actual_signal_counts[actual_signal_counts['signal_actual']!=0]
pred_signal_counts = y['signal_pred'].diff().value_counts()
print(actual_signal_counts)


In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2)

def plot_value_counts(ax, df, col):
    vc = df[col].value_counts()
    label_map={0:'oom', 1:'buy', -1: 'sell'}
    labels = [ label_map[signal] for signal in vc.index.tolist()]
    ax.pie(y['signal_pred'].value_counts(), labels = labels)
    ax.set_title(col)

plot_value_counts(ax1, y, 'signal_pred')
plot_value_counts(ax2, y, 'signal_actual')

plt.show()

#y['signal_pred'].value_counts().plot(kind='pie', title='predicted', labels=['a','b','c'], axis=ax1)
#y['signal_pred'].value_counts().plot(kind='pie', title='predicted', labels=['a','b','c'])

In [ ]:
import seaborn as sns

sns.heatmap(y.corr(), annot=True)
plt.title('Predicted and actual signal')
plt.show()

In [ ]:
from sklearn.metrics import classification_report

clf_report = classification_report(y['signal_actual'],
                                   y['signal_pred'],
                                   #labels=['sell', 'flat', 'buy'],
                                   target_names=['sell', 'flat', 'buy'],
                                   output_dict=True)
#print(clf_report)
#sns.heatmap(pd.DataFrame(clf_report).iloc[:-1, :].T, annot=True)
#sns.heatmap(pd.DataFrame(clf_report).iloc[:-1, :-3].T[::-1], annot=True)
sns.heatmap(pd.DataFrame(clf_report).iloc[:-1, :].T, annot=True)

In [ ]:
from sklearn import metrics
confusion_matrix = metrics.confusion_matrix(y['signal_actual'], y['signal_pred'])

metrics.ConfusionMatrixDisplay(confusion_matrix = confusion_matrix, display_labels = ['Sell', 'Flat', 'Buy']).plot()
plt.title('Confusion matrix')
plt.show()
print(confusion_matrix)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

plt.plot(y.index, y['signal_pred']-y['signal_actual'])
plt.gca().xaxis.set_major_locator(mdates.MonthLocator(interval=5))
plt.xticks(rotation=45)

#ax.format_xdata = mdates.DateFormatter('%Y-%m-%d')
#plt.gca().xaxis.set_major_formatter(ConsizeDateFormatter())
plt.show()

In [ ]:
import plotly.express as px
px.bar(x=y.index, 
    y=y['signal_pred']-y['signal_actual'], 
    title='Predicted and actual signal difference. diff = predicted - actual',
    labels = {'x': 'time', 'y':'predicted - actual'}
      ).show()